In [ ]:
def img_similarity(title, image, description):
    URL_FORMAT = f"https://www.google.com/search?&q={title}&tbm=isch"
    

In [ ]:
import requests
from bs4 import BeautifulSoup
import os

# URL of the web page containing the image
url = f"https://search.brave.com/images?q=amul%20bread"

# Send a GET request to the URL
response = requests.get(url)

# Parse the HTML content
soup = BeautifulSoup(response.text, "html.parser")
print(url)
# Find all image tags
print(soup.prettify())
# main_div = soup.find("img", _class="svelte-hfp50 loaded")
# print(main_div)
# image_tags = soup.find("img", _class="svelte-hfp50 loaded")
# print(image_tags)
# Directory to save the images
save_dir = "images"

# Create the directory if it doesn't exist
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

# Download and save each image
for img in image_tags:
    img_url = img["src"]
    # Download the image
    img_response = requests.get(img_url)
    # Extract the image filename from the URL
    img_filename = os.path.join(save_dir, os.path.basename(img_url))
    # Save the image
    with open(img_filename, "wb") as f:
        f.write(img_response.content)
        print(f"Image '{img_filename}' saved successfully.")

In [ ]:
import requests
import io
from PIL import Image
from selenium.webdriver.common.by import By
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager


def get_images_from_google(url):
    wd = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
    wd.get(url)
    # image_url = None

    thumbnail = wd.find_element(By.CSS_SELECTOR, "Q4LuWd")
    # thumbnail.click()
    # image_url = wd.find_element(By.CLASS_NAME, "iPVvYb")
    # image_url = wd.find_element(By.CLASS_NAME, "sFlh5c pT0Scc iPVvYb")

    print(thumbnail)
    time.sleep(1)
    wd.quit()

    # return imgResults


def download_image(download_path, url, file_name):
    image_content = requests.get(url).content
    image_file = io.BytesIO(image_content)
    image = Image.open(image_file)
    file_path = download_path + file_name

    with open(file_path, "wb") as f:
        image.save(f, "jpeg")

    print("done")


print(get_images_from_google(f"https://search.brave.com/images?q=amul%20bread"))

In [30]:
download_image("./images/", "https://m.media-amazon.com/images/I/71+TmE0ZekL.jpg", "test.jpg")

done


In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity


def preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    return img_array


def extract_features(img_path, model):
    img = preprocess_image(img_path)
    features = model.predict(img)
    return features.flatten()


def calculate_similarity(features1, features2):
    similarity_score = cosine_similarity([features1], [features2])
    return similarity_score[0][0]


# Load pre-trained VGG16 model
model = VGG16(weights="imagenet", include_top=False)

# Path to the images
image1_path = r"src\test_amul_bread.jpeg"
# image2_path = r"src\brit_bread.jpeg"
# image1_path = r"src\brit_bread2.jpeg"
image2_path = r"src\butter_bread_sandwich_1.jpeg"

# Extract features
features1 = extract_features(image1_path, model)
features2 = extract_features(image2_path, model)

# Calculate similarity score
similarity_score = calculate_similarity(features1, features2)
print("Similarity score:", similarity_score)

1/1 [==============================] - 0s 79ms/step
Similarity score: 0.17009449


In [1]:
import tensorflow as tf

# tf.config.list_physical_devices("GPU")

In [2]:
tf.config.list_physical_devices("GPU")

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [27]:
import requests
from bs4 import BeautifulSoup
from PIL import Image
import io
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity



def check_img_similarity(query, product_image):
    def get_ref_img_url(query):
    # Prepare Google search URL
        query = "+".join(query.split())
        url = f"https://www.google.com/search?q={query}&tbm=isch"

        # Send HTTP request and get response
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
        }
        response = requests.get(url, headers=headers)
        response.raise_for_status()

        # Parse the HTML content
        soup = BeautifulSoup(response.text, "html.parser")

        # Extract the URL of the second image
        img_tags = soup.find_all("img")
        img_url = img_tags[1]["src"]
        # if len(img_tags) >= 2:
            # return img_url

        def download_image(download_path, url, file_name):
            image_content = requests.get(url).content
            image_file = io.BytesIO(image_content)
            image = Image.open(image_file)
            file_path = download_path + file_name

            with open(file_path, "wb") as f:
                image.save(f, "jpeg")

            print("done")
            
        download_image("./images/", img_url, "test1.jpg")


    get_ref_img_url(query)
    path1 = product_image
    path2 = "./images/test1.jpg"

    def preprocess_image(img_path):
        img = image.load_img(img_path, target_size=(224, 224))
        img_array = image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array = preprocess_input(img_array)
        return img_array

    def extract_features(img_path, model):
        img = preprocess_image(img_path)
        features = model.predict(img)
        return features.flatten()

    def calculate_similarity(features1, features2):
        similarity_score = cosine_similarity([features1], [features2])
        return similarity_score[0][0]

    # Load pre-trained VGG16 model
    model = VGG16(weights="imagenet", include_top=False)
    features1 = extract_features(path1, model)
    features2 = extract_features(path2, model)

    # Calculate similarity score
    similarity_score = calculate_similarity(features1, features2)
    print("Similarity score:", similarity_score)
    
    return similarity_score

In [28]:
product_image = r"src\brit_bread2.jpeg" #image of britannia bread with brittania reference
check_img_similarity("brittania brown bed", product_image)

done
1/1 [==============================] - 0s 29ms/step
Similarity score: 0.6413834


0.6413834

In [29]:
product_image = r"src\brit_bread2.jpeg" #image of britannia bread with amul bread reference
check_img_similarity("Amul brown bread", product_image)

done
1/1 [==============================] - 0s 41ms/step
Similarity score: 0.20311785


0.20311785